In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import log_loss
from sklearn.svm import LinearSVC,LinearSVR,SVC,SVR
from scipy.stats import norm,skew

# Setting the number of columns to display(that is display all columns)
pd.set_option('display.max_columns', None)
# year
yr=2018



these are the full forms 

WFGM - field goals made (by the winning team)

WFGA - field goals attempted (by the winning team)

WFGM3 - three pointers made (by the winning team)

WFGA3 - three pointers attempted (by the winning team)

WFTM - free throws made (by the winning team)

WFTA - free throws attempted (by the winning team)

WOR - offensive rebounds (pulled by the winning team)

WDR - defensive rebounds (pulled by the winning team)

WAst - assists (by the winning team)

WTO - turnovers committed (by the winning team)

WStl - steals (accomplished by the winning team)

WBlk - blocks (accomplished by the winning team)

WPF - personal fouls committed (by the winning team)

In [2]:
cities=pd.read_csv("WStage2DataFiles/WCities.csv")
game_cities=pd.read_csv("WStage2DataFiles/WGameCities.csv")

tourney_seeds=pd.read_csv("WStage2DataFiles/WNCAATourneySeeds.csv")

regular_season_results=pd.read_csv("WStage2DataFiles/WRegularSeasonCompactResults.csv")
detailed_regular_season=pd.read_csv("WStage2DataFiles/WRegularSeasonDetailedResults.csv")

tourney_results=pd.read_csv("WStage2DataFiles/WNCAATourneyCompactResults.csv")
detailed_tourney=pd.read_csv("WStage2DataFiles/WNCAATourneyDetailedResults.csv")


# regular_season_results=regular_season_results.loc[regular_season_results['Season']<=yr]
# tourney_results=tourney_results.loc[tourney_results['Season']<yr]

detailed_regular_season=detailed_regular_season.loc[detailed_regular_season['Season']<=yr]
detailed_tourney=detailed_tourney.loc[detailed_tourney['Season']<yr]

sample_submission=pd.read_csv("WStage2DataFiles/WSampleSubmissionStage2.csv")

print("Regular season results",regular_season_results.shape,detailed_regular_season.shape)
print("Tourney results",tourney_results.shape,detailed_tourney.shape)

Regular season results (101893, 8) (46342, 34)
Tourney results (1260, 8) (504, 34)


In [3]:
detailed_tourney['Type']=1          # tournament
detailed_regular_season['Type']=0      #regular season
detailed_tourney=pd.concat([detailed_tourney,detailed_regular_season],axis=0,ignore_index=True)
print(detailed_tourney.shape)
detailed_tourney.head()

(46846, 35)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Type
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,1
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,1
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,1
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,1
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12,1


In [4]:
# adding seeds to tournnament results
def extract_seedint(seed):
    k=int(seed[1:])
    return k;
def region(seed):
    return seed[0];
tourney_seeds['seed_int']=tourney_seeds['Seed'].apply(extract_seedint)
tourney_seeds['region']=tourney_seeds['Seed'].apply(region)
tourney_seeds.drop(columns=['Seed'],inplace=True)

winseeds = tourney_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed','region':'WRegion'})
lossseeds = tourney_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed','region':'LRegion'})
df_dummy = pd.merge(left=tourney_results, right=winseeds, how='left', on=['Season', 'WTeamID'])
tourney_results= pd.merge(left=df_dummy, right=lossseeds,how='left',on=['Season', 'LTeamID'])

In [5]:
# the formulae for the possession in the paper is wrong..!!!
detailed_tourney['WPossessions']=detailed_tourney['WFGA']-detailed_tourney['WOR']+detailed_tourney['WTO']+0.475*detailed_tourney['WFTA']
detailed_tourney['LPossessions']=detailed_tourney['LFGA']-detailed_tourney['LOR']+detailed_tourney['LTO']+0.475*detailed_tourney['LFTA']

# in a game the possesion of the two teams are nearly same(+2 or -2). So we can take the average
# the details is given in the link  https://kenpom.com/blog/stats-explained/
detailed_tourney['Possessions']=detailed_tourney['WPossessions']*0.5+detailed_tourney['LPossessions']*0.5

# offensive and defensive effiencies by dividing the effincies with the average number
detailed_tourney['WOE']=(detailed_tourney['WScore']*100)/detailed_tourney['Possessions']
detailed_tourney['WDE']=(detailed_tourney['LScore']*100)/detailed_tourney['Possessions']

detailed_tourney['LOE']=(detailed_tourney['LScore']*100)/detailed_tourney['Possessions']
detailed_tourney['LDE']=(detailed_tourney['WScore']*100)/detailed_tourney['Possessions']

# adding seeds 
df_dummy = pd.merge(left=detailed_tourney, right=winseeds, how='left', on=['Season', 'WTeamID'])
detailed_tourney= pd.merge(left=df_dummy, right=lossseeds,how='left',on=['Season', 'LTeamID'])

detailed_tourney['WSeed'].fillna(0,inplace=True)
detailed_tourney['LSeed'].fillna(0,inplace=True)

# finding the seed difference
detailed_tourney['SeedDiff']=detailed_tourney['WSeed']-detailed_tourney['LSeed']

# dropping the region columns 
detailed_tourney.drop(columns=['WRegion','LRegion'],inplace=True)




print(detailed_tourney.shape)
detailed_tourney.head()

(46846, 45)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Type,WPossessions,LPossessions,Possessions,WOE,WDE,LOE,LDE,WSeed,LSeed,SeedDiff
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,1,65.025,64.375,64.7000,106.646059,85.007728,85.007728,106.646059,4.0,13.0,-9.0
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,1,71.025,70.125,70.5750,94.934467,93.517535,93.517535,94.934467,8.0,9.0,-1.0
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,1,65.450,65.800,65.6250,109.714286,56.380952,56.380952,109.714286,2.0,15.0,-13.0
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,1,72.350,72.450,72.4000,103.591160,84.254144,84.254144,103.591160,3.0,14.0,-11.0
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12,1,59.800,62.075,60.9375,101.743590,68.923077,68.923077,101.743590,5.0,12.0,-7.0


I will calculate the national offensive and defensive effi

May I will use this to find the adjusted offensive and defensive effi. But currently I dont know any formuale for calculating the adjusted effi

In [6]:
# now i have to calculate the adjusted OE,DE for winning team and losing team
# before that I have to calculate the team offensive and defensive efficiency overall games 
woe=pd.DataFrame()
wde=pd.DataFrame()
woe['TeamID']=detailed_tourney.groupby('WTeamID').WOE.sum().index
woe['wOE']=detailed_tourney.groupby('WTeamID').WOE.sum().values

wde['TeamID']=detailed_tourney.groupby('WTeamID').WDE.sum().index
wde['wDE']=detailed_tourney.groupby('WTeamID').WDE.sum().values

loe=pd.DataFrame()
lde=pd.DataFrame()
loe['TeamID']=detailed_tourney.groupby('LTeamID').LOE.sum().index
loe['lOE']=detailed_tourney.groupby('LTeamID').LOE.sum().values

lde['TeamID']=detailed_tourney.groupby('LTeamID').LDE.sum().index
lde['lDE']=detailed_tourney.groupby('LTeamID').LDE.sum().values

oe=pd.merge(left=woe,right=loe,how='outer',on=['TeamID'])
de=pd.merge(left=wde,right=lde,how='outer',on=['TeamID'])

oe.fillna(0,inplace=True)
de.fillna(0,inplace=True)

oe['OE']=oe['wOE']+oe['lOE']
oe.drop(columns=['wOE','lOE'],inplace=True)

de['DE']=de['wDE']+de['lDE']
de.drop(columns=['wDE','lDE'],inplace=True)

# number of games 
wgames=pd.DataFrame()
wgames['TeamID']=detailed_tourney['WTeamID'].value_counts().index
wgames['wcount']=detailed_tourney['WTeamID'].value_counts().values

lgames=pd.DataFrame()
lgames['TeamID']=detailed_tourney['LTeamID'].value_counts().index
lgames['lcount']=detailed_tourney['LTeamID'].value_counts().values


games=pd.merge(left=wgames,right=lgames,how='outer',on=['TeamID'])
games['wcount'].fillna(0,inplace=True)
games['lcount'].fillna(0,inplace=True)
games['number']=games['wcount']+games['lcount']
games.drop(columns=['wcount','lcount'],inplace=True)

oe=pd.merge(left=oe,right=games,how='outer',on=['TeamID'])
de=pd.merge(left=de,right=games,how='outer',on=['TeamID'])

oe['OE']=oe['OE']/oe['number']
de['DE']=de['DE']/de['number']

oe.drop(columns=['number'],inplace=True)
de.drop(columns=['number'],inplace=True)

print(oe.shape,de.shape)

national_oe_average=(oe['OE'].sum())/(oe['OE'].shape[0])
national_de_average=(de['DE'].sum())/(de['DE'].shape[0])
print("The national offensive average is",national_oe_average)
print("The national defensive average is",national_de_average)

(350, 2) (350, 2)
The national offensive average is 90.4326058605
The national defensive average is 90.9552035152


## TRAIN DATA PREPROCESSING

In [7]:
def change(s):
    if s=='N':
        return 0;
    elif s=='H':
        return 1;
    else:
        return -1;

    

df_dummy=pd.merge(left=detailed_tourney
                  ,right=oe.rename(columns={'TeamID':'WTeamID','OE':'WOE_avg'}),
                  how='left',on=['WTeamID'])

detailed_tourney=pd.merge(left=df_dummy
                           ,right=de.rename(columns={'TeamID':'WTeamID','DE':'WDE_avg'})
                           ,how='left',on=['WTeamID'])

df_dummy=pd.merge(left=detailed_tourney,
                  right=oe.rename(columns={'TeamID':'LTeamID','OE':'LOE_avg'}),
                  how='left',on=['LTeamID'])

detailed_tourney=pd.merge(left=df_dummy,
                          right=de.rename(columns={'TeamID':'LTeamID','DE':'LDE_avg'}),
                          how='left',on=['LTeamID'])

# finding the effi margin
detailed_tourney['WEM']=detailed_tourney['WOE_avg']-detailed_tourney['WDE_avg']
detailed_tourney['LEM']=detailed_tourney['LOE_avg']-detailed_tourney['LDE_avg']

# adjusted effi/ This is what i understood from the definition https://kenpom.com/blog/ratings-glossary/
detailed_tourney['adj_WOE']=detailed_tourney['WOE_avg']*(national_oe_average/national_de_average)
detailed_tourney['adj_WDE']=detailed_tourney['WDE_avg']*(national_de_average/national_oe_average)

detailed_tourney['adj_LOE']=detailed_tourney['LOE_avg']*(national_oe_average/national_de_average)
detailed_tourney['adj_LDE']=detailed_tourney['LDE_avg']*(national_de_average/national_oe_average)

# adding the scores
detailed_tourney['Diff']=detailed_tourney['WScore']-detailed_tourney['LScore']


detailed_tourney=detailed_tourney.loc[:,['WLoc','WOE_avg','WDE_avg','LOE_avg','LDE_avg',
                                         'Type','SeedDiff','WEM','LEM','adj_WOE','adj_WDE','adj_LOE','adj_LDE'
                                        ,'Diff']]
detailed_tourney['WLoc']=detailed_tourney['WLoc'].apply(change)
detailed_tourney['Result']=1

temp=pd.DataFrame()

# creating features
temp['WLoc']=-detailed_tourney['WLoc']

temp['WOE_avg']=detailed_tourney['LOE_avg']

temp['WDE_avg']=detailed_tourney['LDE_avg']

temp['LOE_avg']=detailed_tourney['WOE_avg']

temp['LDE_avg']=detailed_tourney['WDE_avg']

temp['Type']=detailed_tourney['Type']

temp['SeedDiff']=-detailed_tourney['SeedDiff']

temp['WEM']=detailed_tourney['LEM']

temp['LEM']=detailed_tourney['WEM']

temp['adj_WOE']=detailed_tourney['adj_LOE']

temp['adj_WDE']=detailed_tourney['adj_LDE']

temp['adj_LOE']=detailed_tourney['adj_WOE']

temp['adj_LDE']=detailed_tourney['adj_WDE']

temp['Diff']=-detailed_tourney['Diff']

temp['Result']=0

games=pd.concat([detailed_tourney,temp],axis=0,ignore_index=True)
games=games.loc[games['Type']==1]
print(games.shape)
games.head()

(1008, 15)


,WLoc,WOE_avg,WDE_avg,LOE_avg,LDE_avg,Type,SeedDiff,WEM,LEM,adj_WOE,adj_WDE,adj_LOE,adj_LDE,Diff,Result
0,0,111.271937,77.720009,96.114439,88.185513,1,-9.0,33.551928,7.928926,110.632606,78.169143,95.562198,88.695125,14,1
1,0,99.026108,86.090117,94.973440,89.799406,1,-1.0,12.935991,5.174034,98.457137,86.587620,94.427755,90.318344,1,1
2,1,101.404878,80.258298,90.907847,79.332841,1,-13.0,21.146580,11.575006,100.822240,80.722099,90.385521,79.791294,35,1
3,1,101.467002,84.254017,92.780566,89.637890,1,-11.0,17.212985,3.142677,100.884007,84.740909,92.247481,90.155895,14,1
4,0,90.869253,88.429263,98.078276,86.436125,1,-7.0,2.439989,11.642151,90.347149,88.940284,97.514751,86.935627,20,1


## TEST PREPREOCESSING

In [8]:
def yr(s):
    yr,team1,team2=map(int,s.split('_'))
    return yr;

def t1(s):
    yr,team1,team2=map(int,s.split('_'))
    return team1;

def t2(s):
    yr,team1,team2=map(int,s.split('_'))
    return team2;

sample_submission['Season']=sample_submission['ID'].apply(yr)
sample_submission['WTeamID']=sample_submission['ID'].apply(t1)
sample_submission['LTeamID']=sample_submission['ID'].apply(t2)


df_dummy=pd.merge(left=sample_submission,
                  right=oe.rename(columns={'TeamID':'WTeamID','OE':'WOE_avg'}),
                  how='left',on=['WTeamID'])

sample_submission=pd.merge(left=df_dummy,
                           right=de.rename(columns={'TeamID':'WTeamID','DE':'WDE_avg'}),
                           how='left',on=['WTeamID'])

df_dummy=pd.merge(left=sample_submission,
                  right=oe.rename(columns={'TeamID':'LTeamID','OE':'LOE_avg'}),
                  how='left',on=['LTeamID'])

sample_submission=pd.merge(left=df_dummy,
                           right=de.rename(columns={'TeamID':'LTeamID','DE':'LDE_avg'}),
                         how='left',on=['LTeamID'])


# adding adjusted 
sample_submission['adj_WOE']=sample_submission['WOE_avg']*(national_oe_average/national_de_average)
sample_submission['adj_WDE']=sample_submission['WDE_avg']*(national_de_average/national_oe_average)

sample_submission['adj_LOE']=sample_submission['LOE_avg']*(national_oe_average/national_de_average)
sample_submission['adj_LDE']=sample_submission['LDE_avg']*(national_de_average/national_oe_average)


#adding effi margin
sample_submission['WEM']=sample_submission['WOE_avg']-sample_submission['WDE_avg']
sample_submission['LEM']=sample_submission['LOE_avg']-sample_submission['LDE_avg']


# adding seeds 
df_dummy = pd.merge(left=sample_submission, right=winseeds, how='left', on=['Season', 'WTeamID'])
sample_submission= pd.merge(left=df_dummy, right=lossseeds,how='left',on=['Season', 'LTeamID'])

sample_submission['WSeed'].fillna(0,inplace=True)
sample_submission['LSeed'].fillna(0,inplace=True)

sample_submission['SeedDiff']=sample_submission['WSeed']-sample_submission['LSeed']
sample_submission.drop(columns=['WRegion','LRegion'],inplace=True)

sample_submission['WLoc']=0
sample_submission['Type']=1

sample_submission.head()

,ID,Pred,Season,WTeamID,LTeamID,WOE_avg,WDE_avg,LOE_avg,LDE_avg,adj_WOE,adj_WDE,adj_LOE,adj_LDE,WEM,LEM,WSeed,LSeed,SeedDiff,WLoc,Type
0,2018_3110_3113,0.5,2018,3110,3113,92.002411,85.894312,94.207494,85.774006,91.473797,86.390683,93.666210,86.269682,6.1081,8.433489,14,7,7,0,1
1,2018_3110_3114,0.5,2018,3110,3114,92.002411,85.894312,93.132736,81.767606,91.473797,86.390683,92.597627,82.240129,6.1081,11.365131,14,14,0,0,1
2,2018_3110_3124,0.5,2018,3110,3124,92.002411,85.894312,111.271937,77.720009,91.473797,86.390683,110.632606,78.169143,6.1081,33.551928,14,2,12,0,1
3,2018_3110_3125,0.5,2018,3110,3125,92.002411,85.894312,96.087765,91.606059,91.473797,86.390683,95.535678,92.135438,6.1081,4.481706,14,12,2,0,1
4,2018_3110_3129,0.5,2018,3110,3129,92.002411,85.894312,93.886949,91.830726,91.473797,86.390683,93.347506,92.361404,6.1081,2.056223,14,16,-2,0,1


## regression

In [9]:
p=['WLoc','adj_WOE','adj_WDE','adj_LOE','adj_LDE','SeedDiff']

X=games.loc[:,p].values
y=games.loc[:,'Diff'].values
X_test=sample_submission.loc[:,p].values

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1,shuffle=True,random_state=42)


print("Training Shape",X_train.shape)
print("Training Label Shape",y_train.shape)
print("Validation Shape",X_val.shape)
print("Validation Label Shape",y_val.shape)
print("Test Shape",X_test.shape)

Training Shape (907, 6)
Training Label Shape (907,)
Validation Shape (101, 6)
Validation Label Shape (101,)
Test Shape (2016, 6)


In [10]:
lr=LinearRegression()
lr.fit(X_train,y_train)
print("training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))

training Score 0.643984609858
Validation Score 0.619695333505


In [11]:
xgb=XGBRegressor()
xgb.fit(X_train,y_train)
print("Training Score",xgb.score(X_train,y_train))
print("validation Score",xgb.score(X_val,y_val))

Training Score 0.756660384222
validation Score 0.610492139646


## predictions

In [12]:
games['Pred_regressor']=lr.predict(X)
sample_submission['Pred_regressor']=lr.predict(X_test)

## TRAINING

In [13]:
# features taken
p=['SeedDiff','Pred_regressor']

X=games.loc[:,p].values
y=games.loc[:,'Result'].values
X_test=sample_submission.loc[:,p].values

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1,shuffle=True,random_state=42)


print("Training Shape",X_train.shape)
print("Training Label Shape",y_train.shape)
print("Validation Shape",X_val.shape)
print("Validation Label Shape",y_val.shape)
print("Test Shape",X_test.shape)

Training Shape (907, 2)
Training Label Shape (907,)
Validation Shape (101, 2)
Validation Label Shape (101,)
Test Shape (2016, 2)


## LOGISTIC REGRESSION

In [14]:
param={'C':[0.001,0.003,0.006,0.01,0.02,0.03,0.04,0.05,0.06,1,3,6,10]}
lr=LogisticRegression()
gridsearch=GridSearchCV(lr,param_grid=param,scoring='neg_log_loss',cv=5)
gridsearch.fit(X_train,y_train)
print("The Best C is",gridsearch.best_params_['C'])

The Best C is 0.003


In [15]:
lr=LogisticRegression(C=gridsearch.best_params_['C'])
lr.fit(X_train,y_train)
print("The Training Log loss is",log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print("The Validation Log loss is",log_loss(y_val,lr.predict_proba(X_val)[:,1]))

The Training Log loss is 0.425675106476
The Validation Log loss is 0.460017806886


In [16]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
print("The Training Log loss is",log_loss(y_train,xgb.predict_proba(X_train)[:,1]))
print("The Validation Log loss is",log_loss(y_val,xgb.predict_proba(X_val)[:,1]))

The Training Log loss is 0.366084036929
The Validation Log loss is 0.507490088831


In [17]:
sample_submission['Pred']=lr.predict_proba(X_test)[:,1]
ans=sample_submission.loc[:,['ID','Pred']]
ans.head()

,ID,Pred
0,2018_3110_3113,0.169989
1,2018_3110_3114,0.447395
2,2018_3110_3124,0.021323
3,2018_3110_3125,0.382833
4,2018_3110_3129,0.616883


In [18]:
ans.to_csv("dont_trust_reg_stage2.csv",index=False)
print("Done")

Done


In [19]:
sample_submission['Pred']=lr.predict_proba(X_test)[:,1]*0.5+xgb.predict_proba(X_test)[:,1]*0.5
ans=sample_submission.loc[:,['ID','Pred']]
ans.head()

,ID,Pred
0,2018_3110_3113,0.165053
1,2018_3110_3114,0.412808
2,2018_3110_3124,0.012367
3,2018_3110_3125,0.356445
4,2018_3110_3129,0.597364


In [20]:
sample_submission.to_csv("dont_trust_reg_lr_xgb_stage2.csv",index=False)
print("Done")

Done
